In [10]:
import requests
import json
import pandas as pd
from IPython.display import HTML, display
from wimbledon.vis.Visualise import DataHandlers

In [2]:
with open('github.token', 'r') as f:
    token = f.read().strip()

headers = {"Authorization": "Bearer " + token}


def run_query(query):  # A simple function to use requests.post to make the API call. Note the json= section.
    request = requests.post('https://api.github.com/graphql', json={'query': query}, headers=headers)
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception("Query failed to run by returning code of {}. {}".format(request.status_code, query))


In [3]:
emojis = {'CONFUSED': '😕',
          'EYES': '👀',
          'HEART': '❤️',
          'HOORAY': '🎉',    
          'ROCKET': '🚀',
          'THUMBS_DOWN': '❌',
          'THUMBS_UP': '✔️',
          'LAUGH': '✅'}

In [4]:
fc = DataHandlers.Forecast()

Enpoint: whoami | Time: 0.5059723990125349 
Enpoint: people/475627 | Time: 0.48271679900062736 

AUTHENTICATED USER:
Ed Chalstrey echalstrey@turing.ac.uk

CLIENTS
Enpoint: clients | Time: 0.5350835980061674 
PROJECTS
Enpoint: projects | Time: 0.7616336270002648 
ROLES
Enpoint: roles | Time: 0.6055478719936218 
PEOPLE
Enpoint: people | Time: 0.5945824310038006 
PLACEHOLDERS
Enpoint: placeholders | Time: 0.48869862699939404 
MILESTONES
Enpoint: milestones | Time: 0.4685551090078661 
ASSIGNMENTS
Enpoint: assignments | Time: 0.7180030739982612 
DONE! (5.2s)


In [5]:
fc.people_totals.resample('MS').mean()  # numbers are already assigned

id,399979,408178,408179,408180,408181,408182,408183,408184,408185,408186,...,458587,458588,458589,460554,472253,474864,475627,478638,478799,479080
2016-11-01,1.00000,1.000000,1.0,1.000000,1.0000,0.00000,1.00000,1.0,0.000000,0.000000,...,1.00,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0
2016-12-01,1.00000,1.000000,1.0,1.000000,1.0000,0.00000,1.00000,1.0,0.000000,0.000000,...,1.00,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0
2017-01-01,1.00000,1.000000,1.0,1.000000,1.0000,0.00000,1.00000,1.0,0.000000,0.000000,...,1.00,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0
2017-02-01,1.00000,1.000000,1.0,1.000000,1.0000,0.00000,1.00000,1.0,0.000000,0.000000,...,1.00,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0
2017-03-01,1.00000,1.000000,1.0,1.000000,1.0000,0.00000,1.00000,1.0,0.000000,0.000000,...,1.00,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0
2017-04-01,1.00000,1.000000,1.0,1.000000,1.0000,1.07875,1.23625,1.0,0.331250,0.610000,...,1.00,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0
2017-05-01,1.00000,1.000000,1.0,1.000000,1.0000,1.07875,0.90750,1.0,0.331250,1.031250,...,1.00,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0
2017-06-01,1.00000,1.000000,1.0,1.000000,1.0000,1.07875,0.90750,1.0,0.822500,0.868750,...,1.00,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0
2017-07-01,1.00000,1.000000,1.0,1.000000,1.0000,1.07875,0.90750,1.0,0.653750,0.823750,...,1.00,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0
2017-08-01,1.00000,1.000000,1.0,1.000000,1.0000,1.07875,0.90750,1.0,0.718750,0.823750,...,1.00,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0


In [36]:
resreqdf = fc.project_resourcereq.resample('MS').mean() # grouped by month and mean taken
resreqdf

id,1684536,1684539,1723251,1723252,1723253,1723254,1723255,1723256,1723257,1723258,...,2061037,2061048,2061076,2061084,2063544,2063583,2065747,2065758,2065776,2150325
2016-11-01,0,0,0,0,0,0,0,0,0,0,...,0,0.0,0.0,0,0,0.0,0,0,0,0
2016-12-01,0,0,0,0,0,0,0,0,0,0,...,0,0.0,0.0,0,0,0.0,0,0,0,0
2017-01-01,0,0,0,0,0,0,0,0,0,0,...,0,0.0,0.0,0,0,0.0,0,0,0,0
2017-02-01,0,0,0,0,0,0,0,0,0,0,...,0,0.0,0.0,0,0,0.0,0,0,0,0
2017-03-01,0,0,0,0,0,0,0,0,0,0,...,0,0.0,0.0,0,0,0.0,0,0,0,0
2017-04-01,0,0,0,0,0,0,0,0,0,0,...,0,0.0,0.0,0,0,0.0,0,0,0,0
2017-05-01,0,0,0,0,0,0,0,0,0,0,...,0,0.0,0.0,0,0,0.0,0,0,0,0
2017-06-01,0,0,0,0,0,0,0,0,0,0,...,0,0.0,0.0,0,0,0.0,0,0,0,0
2017-07-01,0,0,0,0,0,0,0,0,0,0,...,0,0.0,0.0,0,0,0.0,0,0,0,0
2017-08-01,0,0,0,0,0,0,0,0,0,0,...,0,0.0,0.0,0,0,0.0,0,0,0,0


In [51]:
fc.get_person_name(399979)
fc.projects.loc[1684539, "name"] # issue name
fc.projects.loc[1684539, "GitHub"] # issue num

'126'

In [42]:
issues = fc.projects["GitHub"].dropna().values
issues

array(['126', '122', '189', '153', '25', '158', '205', '270', '291', '306',
       '96', '76', '118', '174', '311', '286', '199', '31', '115', '64',
       '132', '46', '27', '106', '183', '50', '98', '190', '93', '243',
       '266', '295', '231', '240', '244', '175', '232', '238', '289',
       '230', '309', '308', '122', '317', '218', '234', '298', '238',
       '297', '229', '233', '302', '265', '237', '59', '176', '301', '149',
       '58', '134', '382', '217', '299', '239', '363', '360', '210', '359',
       '379', '381', '34', '332', '293', '28', '378', '377', '383'], dtype=object)

In [49]:
issues_with_resreq = []
for project_id in resreqdf:
    for resreq in resreqdf[project_id]:
        if resreq > 0:  # if at least one month in the dataframe has a resource requirement of more than 0 FTE
            issue_num = fc.projects.loc[project_id, "GitHub"]
            if issue_num in issues:
                issues_with_resreq.append(issue_num)
            print(issue_num, fc.projects.loc[project_id, "name"], fc.projects.loc[project_id, "start_date"], fc.projects.loc[project_id, "end_date"])
            break

96 Living With Machines 2018-06-01 2022-03-31
311 AIDA-Lloyds 2017-04-01 2019-12-31
183 Probabilistic FEM 2018-12-03 2020-02-29
190 Safe Haven 2018-11-01 2021-03-31
nan Team management 2017-04-01 2022-03-31
231 UQM^3 2018-11-01 2020-08-31
nan Support (Legacy) 2017-04-01 2022-03-31
230 Auto DL 2019-03-19 2019-12-31
308 All in One Cancer imaging optimisation 2019-09-01 2021-08-31
317 Fusion modelling- IDEA 2019-04-01 2021-08-31
218 Uncertainty in Government Modelling 2019-04-01 2020-03-31
234 Interpretable AI for health 2019-10-01 2019-12-31
298 SPF Justice 2019-04-01 2022-03-31
229 Micro-location analytics 2019-04-01 2019-09-30
233 Machine learning for cancer 2019-10-01 2020-04-30
237 Urban systems resilience 2019-10-01 2019-12-31
382 NCSC 2018-01-01 2020-03-31
217 Visualisation 2019-06-01 2019-11-30
299 SPF TPS 2019+ 2019-06-01 2022-03-31
239 Logics for DS 2019-04-01 2020-12-31
363 Medevac Decision Support 2019-09-01 2020-02-29
360 NHS Medication Safety 2019-05-01 2019-11-30
210 UCLH C

In [43]:
query = """
{
  repository(owner:"alan-turing-institute", name:"Hut23") {
    issue(number:X) {
          number
          title
          url

          reactionGroups {
            content
            users(first:20) {
                edges {
                    node {
                        login
                        name
                    }
                }
            }
            }
    }
  }
}
"""

In [44]:
def tabulate_emojis(query_result, issue_num):
    data = [
        ["Person"],
        ["Rating"]]
    def add_to_data(laugh=False):
        for reaction in query_result['data']['repository']['issue']['reactionGroups']:
            if (reaction['content'] == 'LAUGH' and laugh) or (reaction['content'] != 'LAUGH' and not laugh):
                for edge in reaction['users']['edges']:
                    if edge['node']['name']:
                        data[0].append(edge['node']['name'])
                    else:
                        data[0].append(edge['node']['login'])

                    data[1].append(emojis[reaction['content']])
                
    add_to_data(laugh=True)
    add_to_data(laugh=False)

    print(str(issue_num) + ": " + query_result['data']['repository']['issue']['title'])
    display(HTML(
   '<table><tr>{}</tr></table>'.format(
       '</tr><tr>'.join(
           '<td>{}</td>'.format('</td><td>'.join(str(_) for _ in row)) for row in data)
       )
    ))


In [45]:
def query_result_contains_emojis(query_result):
    for reaction in query_result['data']['repository']['issue']['reactionGroups']:
        if len(reaction['users']['edges']) > 0:
            return True
    return False

**TODO**
1. Have queries that work for the other issues not currently working for - WORKS FOR ALL, JUST HAD TO FILTER THOSE WITH NO REACTIONS/EMOJIS
2. Save the real names preferably rather than github id - DONE
3. Incorporate the issues list from github directly - DONE
4. Filter issues by resource required - DONE
5. Using the dates, filter by people's availability. Maybe colour the HTML cells.
2. Use code from this notebook in main Wimbledon planner
3. Create mapping from forecast names to github ids/names

In [50]:
for issue_num in issues_with_resreq:
    modified_query = query.replace("X", str(issue_num))
    result = run_query(modified_query)  # Execute the query
    if query_result_contains_emojis(result):
        tabulate_emojis(result, issue_num)

96: Living with Machines - BL project


Person,Giovanni Colavizza,miguelmorin,Jack Roberts,Sarah Gibson,Kasra Hosseini,nbarlowATI,Camila Rangel Smith,Evelina Gabasova,Tim Hobson,Radka Jersakova,James Robinson,Louise Bowler,Oscar T Giles,Tomas Lazauskas,myyong,Jim Madge
Rating,✅,✅,✅,✅,✅,✔️,✔️,✔️,✔️,✔️,✔️,✔️,✔️,❌,❌,❌


311: AIDA/Lloyds


Person,Camila Rangel Smith,Evelina Gabasova,Sarah Gibson,Louise Bowler,Tim Hobson,Giovanni Colavizza,James Robinson,miguelmorin,Kasra Hosseini,Tomas Lazauskas,myyong
Rating,✅,✔️,✔️,✔️,✔️,✔️,✔️,✔️,✔️,❌,❌


183: Probabilistic FEM - Mark Girolami


...

Person,Ed Chalstrey,Sarah Gibson,James Robinson,Louise Bowler,Jim Madge,Eric Daub,Jack Roberts,nbarlowATI,Evelina Gabasova,Oscar T Giles,Radka Jersakova,Roly Perera,myyong,AshwiniKV,Kasra Hosseini,miguelmorin,Tim Hobson
Rating,✅,✔️,✔️,✔️,✔️,✔️,✔️,✔️,✔️,✔️,✔️,❌,❌,❌,❌,❌,❌


332: Robust, Scalable Sequential Monte Carlo (Adam Johansen & Theo Damoulas)


Person,James Robinson,nbarlowATI,Louise Bowler,Jim Madge,Eric Daub,Ed Chalstrey,James Geddes,Oscar T Giles,miguelmorin,Tomas Lazauskas,Tim Hobson,myyong,Sarah Gibson,Roly Perera,Camila Rangel Smith,Evelina Gabasova,Kasra Hosseini,AshwiniKV,Radka Jersakova
Rating,✅,✅,✔️,✔️,✔️,✔️,✔️,✔️,✔️,✔️,✔️,❌,❌,❌,❌,❌,❌,❌,❌


293: Wrattler 2019-20 - Tomas Petricek


Person,Sarah Gibson
Rating,✅
